In [ ]:
# Initialize libraries and functions
import cv2
import numpy as np
import math
import random
import time as timess
from prettytable import PrettyTable
from IPython.display import clear_output
import socket

serverAddressPort   = ("127.0.0.1", 20001)
bufferSize          = 1024
UDPClientSocket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)

def udp_call(msg_to_send):
    bytesToSend         = str.encode(msg_to_send)
    UDPClientSocket.sendto(bytesToSend, serverAddressPort)
    msgFromServer = UDPClientSocket.recvfrom(bufferSize)
    msg = msgFromServer[0].decode()
    return msg

In [ ]:
# Main Function DTSE For Visualization

#Initialization
play_back_speed=1
resize=1.5
img = np.zeros((700,1400,3), np.uint8)
cv2.namedWindow("Visualisation", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Visualisation", int(img.shape[1]/resize), int(img.shape[0]/resize))
tmp_s_r=20
tmp_s_a=20
aircraft_speed=10
radar_speed=1

time=(10**3)*2 #in 1 nano seconds 

visualize=True
idx=0

Emitter_Track_File={}
full_Emitter_Track_File={}
backup_ageinout_Emitter_Track_File={}
backup_actual_Emitter_Track_File={}
rise_fall_flag={}
serach_angles=[]
sn=3
for i in range(sn):
    serach_angles.append(360*i//sn)
serach_angles.insert(1,10)
serach_angles.insert(len(serach_angles),350)
age_in_PFM ={1:0,
             2:0,
             3:0,
             4:0}
age_out_PFM={1:0,
             2:0,
             3:0,
             4:0}

play_flag=True
# play_flag=False

# Main Loop
idx=0
while(idx<500):
    print('tick number - ',str(idx+1))
    udp_call('tick '+str(idx)).split()
    radar_count = list(map(int,udp_call('radar count').split()))[0]
    receivers_n=list(map(int,udp_call('rwr count').split()))[0]


    #Ontick ------------ start
    #Aircraft
    udp_call('aircraft ontick')
    #Radar
    for i in range(radar_count):
        udp_call('radar ontick '+str(i))
    #RWR
    udp_call('rwr ontick')
    #Ontick ------------ stop

    # Get ------------ start
    #Aircraft
    t_pos = list(map(float,udp_call('aircraft get').split()))
    airc_pos=(int(t_pos[0]),int(t_pos[1]))
    airc_heading=t_pos[2]
    #Radar
    rad_positions=[]
    rad_ids=[]
    for i in range(radar_count):
        t_pos = list(map(float,udp_call('radar get '+str(i)).split()))
        radar_pos=(int(t_pos[0]),int(t_pos[1]))
        rad_id=int(t_pos[2])
        rad_ids.append(rad_id)
        rad_positions.append(radar_pos)
    #RWR
    rwr_get=udp_call('rwr get').split('|')
    tmp_emitter_records=[]
    if(rwr_get[0]!=''):
        for i in range(len(rwr_get)):
            tmp=list(rwr_get[i].split(','))
            for j in range(len(tmp)):
                if(j!=1):
                    if('.' in tmp[j]):
                        tmp[j]=float(tmp[j])
                    else:
                        tmp[j]=int(tmp[j])
                else:
                    tmp[j]=bool(tmp[j])
            tmp_emitter_records.append(tmp)
    # print(tmp_emitter_records)
    # Get ------------ stop
 
    # Set ------------ start
    #Aircraft
    udp_call('aircraft set')
    #Radar
    radar_headings=[]
    for i in range(radar_count):
        heading_angle=int(math.degrees(math.atan2(airc_pos[1]-rad_positions[i][1],airc_pos[0]-rad_positions[i][0])))#+180
        heading_noise_range=0
        radar_head_angle=random.randint(heading_angle-heading_noise_range,heading_angle+heading_noise_range)
        radar_headings.append(radar_head_angle)
        udp_call('radar set '+str(i)+str(' ')+str(radar_head_angle))
    #RWR
    udp_call('rwr set')
    # Set ------------ stop

    #Visualization ------------ start
    #Aircraft
    img = np.zeros(img.shape, np.uint8)
    cv2.circle(img, airc_pos, 5, (0,255,0), -1)
    airc_head=(int(airc_pos[0]+(tmp_s_a*1.5*math.cos(math.radians(airc_heading)))),int(airc_pos[1]+(tmp_s_a*math.sin(math.radians(airc_heading)))))
    cv2.line(img, airc_pos, airc_head, (0,255,0), 4)
    #Radar
    for i in range(radar_count):
        #Visualization of Radar
        cv2.circle(img, rad_positions[i], 7, (255,155,0), -1)
        cv2.putText(img,str(rad_ids[i]),(rad_positions[i][0]+20,rad_positions[i][1]),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1,cv2.LINE_AA)
        cv2.line(img, rad_positions[i], 
                 (int(rad_positions[i][0]+(tmp_s_r*math.cos(math.radians(radar_headings[i])))),
                  int(rad_positions[i][1]+(tmp_s_r*math.sin(math.radians(radar_headings[i]))))), (0,0,255), 2)
    #RWR
    for j in range(receivers_n):
        cv2.line(img, airc_pos, 
                 (int(airc_pos[0]+(tmp_s_a*math.cos(math.radians(airc_heading+(360/receivers_n)*(1+2*j)/2)))),
                  int(airc_pos[1]+(tmp_s_a*math.sin(math.radians(airc_heading+(360/receivers_n)*(1+2*j)/2))))), (0,0,255), 2)
    cv2.circle(img, airc_pos, 5, (0,255,0), -1)
    #Visualization ------------ stop


    #Creating Emitter Track File using Emitter records
    record_set=set()
    etf_set=set()
    all_emitter=set()
    for j in Emitter_Track_File.keys():
        etf_set.add(j)
    for j in full_Emitter_Track_File.keys():
        all_emitter.add(j)
    for j in tmp_emitter_records:
        record_set.add(j[0])
        if(j[0] in full_Emitter_Track_File):
            for k in range(len(j)):
                full_Emitter_Track_File[j[0]][k]=j[k]
            full_Emitter_Track_File[j[0]][-1]+=1
            full_Emitter_Track_File[j[0]][1]=True
        else:
            j[1]=True
            full_Emitter_Track_File[j[0]]=j+[1]
            if(j[0] not in backup_actual_Emitter_Track_File):
                backup_actual_Emitter_Track_File[j[0]]={'started_receiving':[idx],'stopped_receiving':[]}
            else:
                backup_actual_Emitter_Track_File[j[0]]['started_receiving'].append(idx)
        if(j[0] in Emitter_Track_File):
            for k in range(len(j)):
                Emitter_Track_File[j[0]][k]=j[k]
            Emitter_Track_File[j[0]][-1]+=1
            Emitter_Track_File[j[0]][1]=True
        else:
            if(full_Emitter_Track_File[j[0]][-1]>=age_in_PFM[j[0]]):
                j[1]=True
                Emitter_Track_File[j[0]]=j+[1]
                if(j[0] not in backup_ageinout_Emitter_Track_File):
                    backup_ageinout_Emitter_Track_File[j[0]]={'started_receiving':[idx],'stopped_receiving':[]}
                else:
                    backup_ageinout_Emitter_Track_File[j[0]]['started_receiving'].append(idx)
    for j in (all_emitter-record_set):
        full_Emitter_Track_File.pop(j)
        backup_actual_Emitter_Track_File[j]['stopped_receiving'].append(idx)
    for j in (etf_set-record_set):
        if(idx-backup_actual_Emitter_Track_File[j]['stopped_receiving'][-1]+0>=age_out_PFM[j]):
            Emitter_Track_File.pop(j)
            backup_ageinout_Emitter_Track_File[j]['stopped_receiving'].append(idx)
        else:
            Emitter_Track_File[j][-1]+=1
            Emitter_Track_File[j][1]=False

    #RWR Display
    rwr_img = np.zeros((500,500,3), np.uint8)
    rwr_x_coor=rwr_img.shape[0]//2
    rwr_y_coor=rwr_img.shape[1]//2
    for j in range(4):
        cv2.line(rwr_img, (rwr_x_coor,rwr_y_coor), (int(rwr_x_coor*(1+0.05*math.cos(math.radians(j*90)))),int(rwr_y_coor*(1+0.05*math.sin(math.radians(j*90))))), (0,255,0), 1)
    max_j=5
    for j in range(1,max_j):
        off_ang=45
        cv2.putText(rwr_img,str((max_j-j)*10)+'dbm',(int(rwr_x_coor*(1+j/(max_j-1)*math.cos(math.radians(off_ang)))+10),int(rwr_y_coor*(1+j/(max_j-1)*math.sin(math.radians(off_ang))))),cv2.FONT_HERSHEY_SIMPLEX,0.4,(255,255,255),1,cv2.LINE_AA)
        cv2.circle(rwr_img, (rwr_x_coor,rwr_y_coor),int(j*(rwr_x_coor/(max_j-1))), (0,255,0), 1)
        

    # Visualization of ETF
    print('\nEmitter Track File')
    t = PrettyTable(['EmitterID','Receiving','Frequency','Angle','Recvied_power','Time'])
    for i in Emitter_Track_File.values():
        t.add_row(i)
        rec_pwr=(((max_j-1)*10)-((i[4] if i[4]<(max_j*10) else (max_j*10))-10))/((max_j-1)*10)
        cv2.circle(rwr_img, (int(rwr_x_coor*(1+rec_pwr*math.cos(math.radians(i[3]-90)))),int(rwr_y_coor*(1+rec_pwr*math.sin(math.radians(i[3]-90))))),5, (0,0,255), -1)
        cv2.putText(rwr_img,str(i[0]),(int(rwr_x_coor*(1+rec_pwr*math.cos(math.radians(i[3]-90)))+10),int(rwr_y_coor*(1+rec_pwr*math.sin(math.radians(i[3]-90))))),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1,cv2.LINE_AA)
    print(t)

    # print('Reception record')
    # for i in backup_ageinout_Emitter_Track_File.keys():
    #     print(str(i)+',')
    #     for j in backup_ageinout_Emitter_Track_File[i].keys():
    #         print('  ','{:17}'.format(j),'-',backup_ageinout_Emitter_Track_File[i][j])



    # print('\nEmitter Track File with age_in and age_out as 0')
    # t = PrettyTable(['EmitterID','Receiving','Frequency','Angle','Recvied_power','Time'])
    # for i in full_Emitter_Track_File.values():
    #     t.add_row(i)
    # print(t)

    # print('\nReception record with age_in and age_out as 0')
    # for i in backup_actual_Emitter_Track_File.keys():
    #     print(str(i)+',')
    #     for j in backup_actual_Emitter_Track_File[i].keys():
    #         print('  ','{:17}'.format(j),'-',backup_actual_Emitter_Track_File[i][j])


    image = cv2.putText(img,'tick number - '+str(idx),(img.shape[1]*6//10,img.shape[0]*1//10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
    cv2.imshow('Visualisation',img)
    cv2.imshow('RWR_Display',rwr_img)
    idx+=1
    if(play_flag):
        if(idx<15 or (idx>92 and idx<98) or (idx>99 and idx<107)):
            # quit_k = cv2.waitKey(0) & 0xFF
            quit_k = cv2.waitKey(play_back_speed) & 0xFF
        else:
            # quit_k = cv2.waitKey(0) & 0xFF
            quit_k = cv2.waitKey(play_back_speed) & 0xFF
    else:
        quit_k = cv2.waitKey(0) & 0xFF
    clear_output(wait=True)
    if quit_k == ord('q'):
        break
    if quit_k == ord('p'):
        play_flag=False
    if quit_k == ord('l'):
        play_flag=True

cv2.imwrite('Visualisation.jpg',img)
cv2.imwrite('RWR_Display.jpg',rwr_img)
UDPClientSocket.sendto(str.encode('stop'), serverAddressPort)
cv2.destroyAllWindows()